In [1]:
from tensorflow import keras

C:\Users\Dell\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from keras.utils import to_categorical

In [3]:
from keras.preprocessing.image import load_img

In [4]:
from keras.models import Sequential

In [5]:
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D

In [6]:
import os
import pandas as pd
import numpy as np

In [12]:
Train_dic='C:\\Users\\Dell\\Face Emotion detection\\images\\train'
Test_dic='C:\\Users\\Dell\\Face Emotion detection\\images\\test'

In [13]:
def createDataframe(dir):
    image_path=[]
    labels=[]
    for label in os.listdir(dir):
        for imageName in os.listdir(os.path.join(dir,label)):
            image_path.append(os.path.join(dir,label,imageName))
            labels.append(label)
        print(label,"Completed")
    return image_path,labels

In [14]:
train=pd.DataFrame()

In [15]:
train['image'],train['label']=createDataframe(Train_dic)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [16]:
train

,image,label
0,C:\Users\Dell\Face Emotion detection\images\tr...,angry
1,C:\Users\Dell\Face Emotion detection\images\tr...,angry
2,C:\Users\Dell\Face Emotion detection\images\tr...,angry
3,C:\Users\Dell\Face Emotion detection\images\tr...,angry
4,C:\Users\Dell\Face Emotion detection\images\tr...,angry
...,...,...
28816,C:\Users\Dell\Face Emotion detection\images\tr...,surprise
28817,C:\Users\Dell\Face Emotion detection\images\tr...,surprise
28818,C:\Users\Dell\Face Emotion detection\images\tr...,surprise
28819,C:\Users\Dell\Face Emotion detection\images\tr...,surprise


In [17]:
test=pd.DataFrame()
test['image'],test['label']=createDataframe(Test_dic)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [18]:
test

,image,label
0,C:\Users\Dell\Face Emotion detection\images\te...,angry
1,C:\Users\Dell\Face Emotion detection\images\te...,angry
2,C:\Users\Dell\Face Emotion detection\images\te...,angry
3,C:\Users\Dell\Face Emotion detection\images\te...,angry
4,C:\Users\Dell\Face Emotion detection\images\te...,angry
...,...,...
7061,C:\Users\Dell\Face Emotion detection\images\te...,surprise
7062,C:\Users\Dell\Face Emotion detection\images\te...,surprise
7063,C:\Users\Dell\Face Emotion detection\images\te...,surprise
7064,C:\Users\Dell\Face Emotion detection\images\te...,surprise


In [20]:
def extract_feature(images):
    features=[]
    for image in tqdm(images):
        img=load_img(image,grayscale=True)
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features

In [21]:
from tqdm.notebook import tqdm

In [22]:
train_feature=extract_feature(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\Dell\anaconda3\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [23]:
test_feature=extract_feature(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

C:\Users\Dell\anaconda3\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [24]:
x_train=train_feature/255.0
x_test=test_feature/255.0

In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
le=LabelEncoder()

In [27]:
le.fit(train['label'])

LabelEncoder()

In [28]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [29]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [30]:
#model creating 
model=Sequential()
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# fully connected layers

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))

#model layer 

model.add(Dense(7,activation='softmax'))

In [31]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [34]:
#model.fit(x=x_train,y=y_train,batch_size=128,epochs=2,validation_data=(x_test,y_test))

In [35]:
from keras.models import model_from_json

In [36]:
json_file=open("facialemotionmodel.json",'r')
model_json=json_file.read()
json_file.close()
model=model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")

In [57]:
label=['angry','disgust','fear','happy','neutral','sad','surprise']

In [58]:
def ef(image):
    img=load_img(image,grayscale=True)
    feature=np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0

In [59]:
image='images/train/sad/130.jpg'
print("original image is of angry ")
img =ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print('model prediction is ',pred_label)

original image is of angry 
1/1 [==============================] - 0s 57ms/step
model prediction is  sad
